# Подготовка датасетов

In [0]:
import numpy as np
import pandas as pd 
import tqdm
import gensim
import os
import pickle
import json

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
PATH_TO_DATA = '/content/drive/My Drive/MusicRecommenderDSMinor/'

### music lists
- подготовка для ворд2век

In [0]:
with open(os.path.join(PATH_TO_DATA,'data/music_lists.csv'), 'w') as f:
  # chunks - удобно для обработки больших по размеру датасетов
  for chunk in pd.read_csv(os.path.join(PATH_TO_DATA, 'data/dataset_anon.csv'), 
                           header=None, iterator=True, chunksize=100000,
                           lineterminator='\n', error_bad_lines=False):
    # присваиваем названия столбцам
    chunk.columns = ['user', 'song', 'artist']
    chunk['artist'] = chunk['artist'].astype(str)
    # group & join
    chunk = chunk.groupby('user')['artist'].apply(','.join).reset_index()
    
    for row in chunk.iterrows():
      f.write(row[1]['artist'] + '\n')

### Исполнители

In [0]:
with open(os.path.join(PATH_TO_DATA, 'data/artists.csv'), 'w') as f:
  for chunk in pd.read_csv(os.path.join(PATH_TO_DATA, 'data/dataset_anon.csv'), 
                           header=None, iterator=True, chunksize=100000,
                           lineterminator='\n', error_bad_lines=False):
    # присв. названия столбцам
    chunk.columns = ['user', 'song', 'artist']
    chunk['artist'] = chunk['artist'].astype(str)
    # считаемся
    chunk = chunk['artist'].value_counts()
    chunk = chunk.reset_index()
    
    # TO fix in next preprocessing stages
    for row in chunk.iterrows():
      f.write(str(row[1][0]) + ',' + str(row[1][1]) + '\n')

In [0]:
with open(os.path.join(PATH_TO_DATA, 'data/artists_agg.csv'), 'w') as f:
  artists = pd.read_csv(os.path.join(PATH_TO_DATA, 'data/artists.csv'), error_bad_lines=False, header=None)
  artists.columns = ['artists', 'count']  
  artists[['count']] = artists[['count']].convert_objects(convert_numeric=True)
  art_agg = artists.groupby(['artists'])['count'].agg('sum')
  art_agg.to_csv(f, header=['artists', 'count'])

### Пользователи
Тут считаем у кого сколько айтемов

In [0]:
with open(os.path.join(PATH_TO_DATA, 'data/users_count.csv'), 'a') as f:
  for chunk in pd.read_csv(os.path.join(PATH_TO_DATA, 'data/dataset_anon.csv'), 
                           iterator=True, chunksize=100000, header=None, lineterminator='\n',
                           error_bad_lines=False):
    # присв. названия столбцам
    chunk.columns = ['user', 'song', 'artist']
    # группируемся считаемся
    chunk = chunk['user'].value_counts()
    # записываем в df_tmp как датафрейм & добавляем ('a')
    df_tmp = pd.DataFrame(chunk.reset_index())
    df_tmp.to_csv(f, header=False)